In [ ]:
import os
from flask import Flask, request, jsonify, send_file
from flask_cors import CORS
import param
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.document_loaders import TextLoader
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain_openai import ChatOpenAI
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import PyPDFLoader
import requests
from config import *

# Define the Flask app
app = Flask(__name__)
CORS(app)  # Apply CORS to the entire app

# Define the load_db function
def load_db(folder_path, chain_type, k):
    documents = []

    for filename in os.listdir(folder_path):
        if filename.endswith(".pdf"):
            file_path = os.path.join(folder_path, filename)
            loader = PyPDFLoader(file_path)
            documents.extend(loader.load())

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
    docs = text_splitter.split_documents(documents)

    api_key = OPENAI_API_KEY
    persist_directory = PERSIST_DIRECTORY
    embedding_function = OpenAIEmbeddings(openai_api_key=api_key)
    db = Chroma.from_documents(docs, embedding_function)
    # db = DocArrayInMemorySearch.from_documents(docs, embeddings)
    retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": k})
    llm_name = MODEL_NAME
    qa = ConversationalRetrievalChain.from_llm(
        llm=ChatOpenAI(model_name=llm_name, temperature=0, openai_api_key=api_key), 
        chain_type=chain_type, 
        retriever=retriever, 
        return_source_documents=True,
        return_generated_question=True,
    )
    return qa

# Define the cbfs class and related functions
class cbfs(param.Parameterized):
    chat_history = param.List([])
    answer = param.String("")
    db_query  = param.String("")
    db_response = param.List([])
    
    def __init__(self, **params):
        super(cbfs, self).__init__( **params)
        self.loaded_folder = "docs/Chat"
        self.qa = load_db(self.loaded_folder, "stuff", 4)
        self.chat_history = []
        self.answer = ""
        self.db_query = ""
        self.db_response = []

    def call_load_db(self, count):
        if count == 0 or file_input.value is None:
            return f"Loaded File: {self.loaded_file}"
        else:
            file_input.save("temp.pdf")
            self.loaded_file = file_input.filename
            self.qa = load_db("temp.pdf", "stuff", 4)
        self.clr_history()
        return f"Loaded File: {self.loaded_file}"

    def convchain(self, query, chat_history, qa):
        if not query:
             return {"User": "", "ChatBot": ""}

        result = qa({"question": query, "chat_history": chat_history})
        chat_history.extend([(query, result["answer"])])
        db_query = result["generated_question"]
        db_response = result["source_documents"]
        answer = result['answer']
    
        db_response_serializable = []
        for doc in db_response:
            doc_dict = {
            "title": doc.title if hasattr(doc, "title") else "Title not available",
            "page_content": doc.page_content if hasattr(doc, "page_content") else ""
        }
            db_response_serializable.append(doc_dict)

        return {
        "User": query,
        "ChatBot": answer,
        "db_query": db_query,
        "db_response": db_response_serializable,
        "db_responses": [doc.__dict__ for doc in db_response]
    }


    def clr_history(self, count=0):
        self.chat_history = []

# Instantiate the cbfs class
cb = cbfs()

@app.route("/ask", methods=["POST"])
def ask():
    data = request.get_json()
    query = data["query"]
    chat_history = []

    response = cb.convchain(query, chat_history, cb.qa)

    response["query"] = query

    # Send the response to the Node.js server using requests.post()
    nodejs_server_url = NODEJS_SERVER_URL
    nodejs_response = requests.post(nodejs_server_url, json=response)  # Use requests.post() here

    print("Response from convchain:", response)

    return jsonify(response)
    data = request.get_json()
    print("Received Data:", data)  # Add this line to print the received data
    if not data or "files" not in data:
        return jsonify({"error": "No files provided."}), 400

    os.makedirs(target_directory, exist_ok=True)
    responses = []

    for file_info in data["files"]:
        file_content = file_info.get("content", "")
        if not file_content:
            responses.append({"error": "No file content provided."})
            continue

        file_name = os.path.basename(file_info["filePath"])
        file_path = os.path.join(target_directory, file_name)
        
        print("file_name",file_name);
        print("file_path",file_path);

        try:
            with open(file_path, "wb") as f:  # Use "wb" for binary write mode
                f.write(file_content)

            responses.append({"message": f"File '{file_name}' uploaded successfully."})
        except Exception as e:
            responses.append({"error": f"Error uploading file '{file_name}': {str(e)}"})

    return jsonify(responses), 200

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000)
